In [19]:
import sys
sys.path.append("/Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/geomeppy/_scripts")

from ladybug.sql import SQLiteResult

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import plotly.express as px

from outputs.sql import SQLReader

from geomeppy import IDF
from geometry.geometry_parser import GeometryParser

In [21]:
ENERGY_PLUS_LOCATION = "../../../../../Applications/EnergyPlus-22-2-0"
IDD_FILE = f"{ENERGY_PLUS_LOCATION}/Energy+.idd"
IDF.setiddname(IDD_FILE)

In [12]:
SQL_PATH = "cases/test03/results/eplusout.sql"
sql = SQLReader(SQL_PATH)
sql


In [13]:
sql.get_zone_data(2)
sql.get_zone_data(3)

In [14]:
sql.prepare_spatial_data([4,8,12])

In [17]:
sql.spatial_data

{4: [['BLOCK NORTH STOREY 0', 30.331, 'rgb(48, 18, 59)'],
  ['BLOCK WEST STOREY 0', 31.073, 'rgb(252, 163, 47)']],
 8: [['BLOCK NORTH STOREY 0', 30.479, 'rgb(70, 115, 234)'],
  ['BLOCK WEST STOREY 0', 31.309, 'rgb(180, 27, 1)']],
 12: [['BLOCK NORTH STOREY 0', 30.472, 'rgb(69, 110, 228)'],
  ['BLOCK WEST STOREY 0', 31.39, 'rgb(122, 4, 2)']]}

In [16]:
sql.prepare_spatial_colors()

In [ ]:
sql.spatial_times

[21 Jul 01:00, 21 Jul 02:00, 21 Jul 03:00]

In [22]:
IDF_PATH = "/Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/geomeppy/cases/test03/out.idf"
idf = IDF(IDF_PATH)

In [23]:
geo = GeometryParser(idf)

In [24]:
geo.get_zones()

In [25]:
geo.zones

[Zone(Block North Storey 0), Zone(Block West Storey 0)]